# Viewing AIS data

This notebook is work in progress that declares a Panel dashboard to visualize AIS data. All available AIS data is visualized with datashader and a `DatetimeInput` panel widget allows display of vessel locations at a chosen time.

Not yet optimized and filtering vessels can take a few moments.

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import datetime as dt
import holoviews as hv
from holoviews.operation.datashader import rasterize
hv.extension('bokeh')

In [ ]:
zone1 = pd.read_csv('./data/AIS_2017_01_Zone01.csv', parse_dates=[1])
zone2 = pd.read_csv('./data/AIS_2017_01_Zone02.csv', parse_dates=[1])
zone3 = pd.read_csv('./data/AIS_2017_01_Zone03.csv', parse_dates=[1])
zones = pd.concat([zone1,zone2, zone3])

In [ ]:
zones.head() # Dataframe structure

In [ ]:
vessels = {name:df.drop_duplicates().sort_values(by='BaseDateTime').set_index('BaseDateTime') for name,df in zones.groupby('VesselName')}

In [ ]:
# NOTE! Some of the data is missing (presumably an unknown vessel)
# Consider substituting MMSI for VesselName if missing
zone1.iloc[8859]

In [ ]:
print("Expected Latitude range %.3f to %.3f"% (min(zones['LAT']), max(zones['LAT'])))
print("Expected Longitude range %.3f to %.3f " % (min(zones['LON']), max(zones['LON'])))

In [ ]:
eastings, northings = zip(*[hv.util.transform.lon_lat_to_easting_northing(lon, lat) for lon, lat 
                          in zip(zones['LON'], zones['LAT'])])

In [ ]:
dt_input = pn.widgets.DatetimeInput(name='Datetime', value=zones['BaseDateTime'].min())
dt_input

In [ ]:
def vessel_at_time(vessel_name, time, vessels):
    df = vessels[vessel_name].drop_duplicates()
    if time < df.index[0]:
        return None # Query before first value
    if time > df.index[-1]:
        return None # Query after last value
    try:
        idx = df.index.get_loc(time, method='nearest')
        return df.iloc[idx]
    except:
        return None

def mark_vessels(value):
    records = []
    empty = {'easting':0., 'northing':-7.081154551613623e-10, 'name':'', 'callsign':''}
    for vessel in vessels.keys():
        match = vessel_at_time(vessel, value, vessels)
        if match is not None:
            easting, northing = hv.util.transform.lon_lat_to_easting_northing(match['LON'], match['LAT'])
            records.append({'easting':easting, 'northing':northing, 
                            'name':match['VesselName'], 'callsign':match['CallSign']})
    markers = pd.DataFrame(records if len(records) != 0 else [empty]) 
    alpha = 1 if len(records) else 0
    return hv.Points(markers, ['easting', 'northing'], ['name','callsign']).opts(color='white', size=4, 
                                                                                 marker='triangle', alpha=alpha)

In [ ]:
points = rasterize(hv.Points(pd.DataFrame({'northing':northings, 
                                           'easting':eastings}), ['easting', 'northing']))
tiles = hv.element.tiles.ESRI().redim(x='easting', y='northing')

overlay = (tiles * points.opts(cmap='fire', width=900, height=500, cnorm='eq_hist', alpha=0.5) 
 * hv.DynamicMap(mark_vessels, streams=[dt_input.param.value]))
message = ("Example times to compare: 2017-01-17 00:00:00 vs 2017-01-18 00:00:00 with "
           "white triangles marking vessel locations. Unoptimized first cut: filtering updates can take a few moments.")
pn.Column('# AIS Data (Work in progress)', message, dt_input, overlay).servable()